In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=49869 sha256=d96d8b4646392014b8f6d81c2512c466f03ab6305ff9720aa0e97c1e82cd01da
  Stored in directory: /tmp/pip-ephem-wheel-cache-gfmdfufa/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

In [6]:
from spuco.group_inference import JTTInference
from spuco.models import MLP
from spuco.util import Trainer
from torch.optim import SGD

trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

jtt = JTTInference(trainset, trainer=trainer, num_epochs=1)

In [7]:
group_partition = jtt.infer_groups()

100%|██████████| 938/938 [00:00<00:00, 1166.35batch/s]


In [8]:
for key in group_partition.keys():
    print(key, len(group_partition[key]))

(0, 0) 59699
(0, 1) 301


In [10]:
from spuco.utils import GroupLabeledDataset

group_labeled_trainset = GroupLabeledDataset(trainset, group_partition)

In [12]:
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

In [13]:
from torch.optim import SGD
from spuco.models import MLP
from spuco.invariant_train import CustomSampleERM


jtt_train = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    indices=group_partition[(0,0)] + 200 * group_partition[(0,1)],
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [14]:
jtt_train.train()

Epoch 0: 100%|██████████| 1874/1874 [00:06<00:00, 292.36batch/s, accuracy=92.5925925925926%, loss=0.263]


In [15]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 96.21749408983452
Group (0, 1) Test Accuracy: 26.47754137115839
Group (0, 2) Test Accuracy: 63.829787234042556
Group (0, 3) Test Accuracy: 58.392434988179666
Group (0, 4) Test Accuracy: 67.37588652482269
Group (1, 0) Test Accuracy: 74.5721271393643
Group (1, 1) Test Accuracy: 98.0440097799511
Group (1, 2) Test Accuracy: 61.76470588235294
Group (1, 3) Test Accuracy: 50.98039215686274
Group (1, 4) Test Accuracy: 26.715686274509803
Group (2, 0) Test Accuracy: 68.8
Group (2, 1) Test Accuracy: 27.733333333333334
Group (2, 2) Test Accuracy: 90.4
Group (2, 3) Test Accuracy: 32.8
Group (2, 4) Test Accuracy: 2.6737967914438503
Group (3, 0) Test Accuracy: 39.447236180904525
Group (3, 1) Test Accuracy: 59.69773299748111
Group (3, 2) Test Accuracy: 67.75818639798489
Group (3, 3) Test Accuracy: 96.22166246851386
Group (3, 4) Test Accuracy: 42.56926952141058
Group (4, 0) Test Accuracy: 48.36272040302267
Group (4, 1) Test Accuracy: 48.36272040302267
Group (4, 2) Test Accur

{(0, 0): 96.21749408983452,
 (0, 1): 26.47754137115839,
 (0, 2): 63.829787234042556,
 (0, 3): 58.392434988179666,
 (0, 4): 67.37588652482269,
 (1, 0): 74.5721271393643,
 (1, 1): 98.0440097799511,
 (1, 2): 61.76470588235294,
 (1, 3): 50.98039215686274,
 (1, 4): 26.715686274509803,
 (2, 0): 68.8,
 (2, 1): 27.733333333333334,
 (2, 2): 90.4,
 (2, 3): 32.8,
 (2, 4): 2.6737967914438503,
 (3, 0): 39.447236180904525,
 (3, 1): 59.69773299748111,
 (3, 2): 67.75818639798489,
 (3, 3): 96.22166246851386,
 (3, 4): 42.56926952141058,
 (4, 0): 48.36272040302267,
 (4, 1): 48.36272040302267,
 (4, 2): 58.4382871536524,
 (4, 3): 56.06060606060606,
 (4, 4): 97.97979797979798}

In [16]:
evaluator.worst_group_accuracy

((2, 4), 2.6737967914438503)

In [17]:
evaluator.average_accuracy

95.61945609346313

In [18]:
evaluator.evaluate_spurious_task()

46.16